# PySpark Huggingface Inferencing
## Conditional generation with Tensorflow

From: https://huggingface.co/docs/transformers/model_doc/t5

### Using TensorFlow

In [2]:
from transformers import AutoTokenizer, TFT5ForConditionalGeneration

2024-10-03 00:58:33.897402: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-03 00:58:33.904044: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-03 00:58:33.911699: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-03 00:58:33.914094: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-03 00:58:33.919757: I tensorflow/core/platform/cpu_feature_guar

In [3]:
import tensorflow as tf

# Enable GPU memory growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)
        
print(tf.__version__)

2.17.0


In [4]:
tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-small")
model = TFT5ForConditionalGeneration.from_pretrained("google-t5/t5-small")

max_source_length = 512
max_target_length = 128

task_prefix = "translate English to German: "

lines = [
    "The house is wonderful",
    "Welcome to NYC",
    "HuggingFace is a company"
]

input_sequences = [task_prefix + l for l in lines]

2024-10-03 00:58:35.458146: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46032 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:01:00.0, compute capability: 8.6
All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [5]:
input_ids = tokenizer(input_sequences, 
                      padding="longest", 
                      max_length=max_source_length,
                      return_tensors="tf").input_ids
outputs = model.generate(input_ids)

/home/rishic/anaconda3/envs/spark-dl-tf/lib/python3.11/site-packages/transformers/generation/tf_utils.py:837: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length.  recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
I0000 00:00:1727917116.671968 1211184 service.cc:146] XLA service 0x7598c0002e20 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1727917116.671985 1211184 service.cc:154]   StreamExecutor device (0): NVIDIA RTX A6000, Compute Capability 8.6
2024-10-03 00:58:36.673983: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-10-03 00:58:36.682170: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907
I0000 00:00:1727917116.705763 1211184 device_compiler.h:188] Compiled cluster using XLA!  This

In [6]:
[tokenizer.decode(o, skip_special_tokens=True) for o in outputs]

['Das Haus ist wunderbar',
 'Willkommen in NYC',
 'HuggingFace ist ein Unternehmen']

In [7]:
model.framework

'tf'

## PySpark

In [8]:
import os
from pathlib import Path
from datasets import load_dataset

In [9]:
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark import SparkConf

In [ ]:
conda_env = os.environ.get("CONDA_PREFIX")

conf = SparkConf()
conf.set("spark.task.maxFailures", "1")
conf.set("spark.driver.memory", "8g")
conf.set("spark.executor.memory", "8g")
conf.set("spark.pyspark.python", f"{conda_env}/bin/python")
conf.set("spark.pyspark.driver.python", f"{conda_env}/bin/python")
conf.set("spark.sql.execution.pyspark.udf.simplifiedTraceback.enabled", "false")
conf.set("spark.sql.pyspark.jvmStacktrace.enabled", "true")
conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
conf.set("spark.python.worker.reuse", "true")
# Create Spark Session
spark = SparkSession.builder.appName("spark-dl-examples").config(conf=conf).getOrCreate()
sc = spark.sparkContext

In [11]:
# load IMDB reviews (test) dataset
data = load_dataset("imdb", split="test")

In [12]:
lines = []
for example in data:
    lines.append([example["text"].split(".")[0]])

len(lines)

25000

### Create PySpark DataFrame

In [13]:
df = spark.createDataFrame(lines, ['lines']).repartition(10)
df.schema

StructType([StructField('lines', StringType(), True)])

In [14]:
df.take(1)

[Row(lines="i do not understand at all why this movie received such good grades from critics - - i've seen tens of documentaries (on TV) about the wine world which were much much better when (if) you watch it, please think of two very annoying aspects of mondovino : first, the filming is just awful and terrible and upsetting : to me, it looked like the guy behind the camera just received the material and was playing with it : plenty of zooms (for no purpose other than pushing the button in/out) for instance - - i almost stopped to watch it because of that ! secondly, the interviewer (the director i think) is not really relevant : he looks like and ask questions like a boy scout, not like a journalist, even if the general idea and themes would have been interesting, too bad conclusion: overrated documentary, maybe only for guys who do not know nothing about wine => not recommended at all (2/10)")]

### Save the test dataset as parquet files

In [15]:
df.write.mode("overwrite").parquet("imdb_test")

### Check arrow memory configuration

In [16]:
spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "512")
# This line will fail if the vectorized reader runs out of memory
assert len(df.head()) > 0, "`df` should not be empty"

## Inference using Spark DL API (PyTorch)
Note: you can restart the kernel and run from this point to simulate running in a different node or environment.

In [17]:
import pandas as pd
from pyspark.ml.functions import predict_batch_udf
from pyspark.sql.functions import col, pandas_udf, struct
from pyspark.sql.types import StringType

In [18]:
# only use first sentence and add prefix for conditional generation
def preprocess(text: pd.Series, prefix: str = "") -> pd.Series:
    @pandas_udf("string")
    def _preprocess(text: pd.Series) -> pd.Series:
        return pd.Series([prefix + s.split(".")[0] for s in text])
    return _preprocess(text)

In [19]:
# only use first N examples, since this is slow
df = spark.read.parquet("imdb_test").limit(100)
df.show(truncate=120)
df.count()

+------------------------------------------------------------------------------------------------------------------------+
|                                                                                                                   lines|
+------------------------------------------------------------------------------------------------------------------------+
|                                       This is so overly clichéd you'll want to switch it off after the first 45 minutes|
|                                                                                   I was very disappointed by this movie|
|                                                                             I think vampire movies (usually) are wicked|
|                           Though not a complete waste of time, 'Eighteen' really wasn't all sweet as it pretended to be|
|This film did well at the box office, and the producers of this mess thought the stars had such good chemistry in thi...|
|               

100

In [20]:
# only use first 100 rows, since generation takes a while
df1 = df.withColumn("input", preprocess(col("lines"), "Translate English to German: ")).select("input").limit(100).cache()

In [21]:
df1.count()

100

In [22]:
df1.show(truncate=120)

+------------------------------------------------------------------------------------------------------------------------+
|                                                                                                                   input|
+------------------------------------------------------------------------------------------------------------------------+
|          Translate English to German: This is so overly clichéd you'll want to switch it off after the first 45 minutes|
|                                                      Translate English to German: I was very disappointed by this movie|
|                                                Translate English to German: I think vampire movies (usually) are wicked|
|Translate English to German: Though not a complete waste of time, 'Eighteen' really wasn't all sweet as it pretended ...|
|Translate English to German: This film did well at the box office, and the producers of this mess thought the stars h...|
|               

In [23]:
def predict_batch_fn():
    import tensorflow as tf
    import numpy as np
    from transformers import TFT5ForConditionalGeneration, AutoTokenizer

    # Enable GPU memory growth
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
        except RuntimeError as e:
            print(e)

    model = TFT5ForConditionalGeneration.from_pretrained("google-t5/t5-small")
    tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-small")

    def predict(inputs):
        flattened = np.squeeze(inputs).tolist()   # convert 2d numpy array of string into flattened python list
        input_ids = tokenizer(flattened, 
                              padding="longest", 
                              max_length=128,
                              return_tensors="tf").input_ids
        output_ids = model.generate(input_ids)
        string_outputs = np.array([tokenizer.decode(o, skip_special_tokens=True) for o in output_ids])
        print("predict: {}".format(len(flattened)))
        return string_outputs
    
    return predict

In [24]:
generate = predict_batch_udf(predict_batch_fn,
                             return_type=StringType(),
                             batch_size=10)

In [25]:
%%time
# first pass caches model/fn
preds = df1.withColumn("preds", generate(struct("input")))
results = preds.collect()

CPU times: user 8.8 ms, sys: 5.99 ms, total: 14.8 ms
Wall time: 11.4 s


In [26]:
%%time
preds = df1.withColumn("preds", generate("input"))
results = preds.collect()

CPU times: user 6.26 ms, sys: 2.86 ms, total: 9.12 ms
Wall time: 11.5 s


In [27]:
%%time
preds = df1.withColumn("preds", generate(col("input")))
results = preds.collect()

CPU times: user 4.97 ms, sys: 5.28 ms, total: 10.2 ms
Wall time: 11.3 s


In [28]:
preds.show(truncate=60)

+------------------------------------------------------------+------------------------------------------------------------+
|                                                       input|                                                       preds|
+------------------------------------------------------------+------------------------------------------------------------+
|Translate English to German: This is so overly clichéd yo...|   Das ist so übertrieben klischeehaft, dass Sie es nach den|
|Translate English to German: I was very disappointed by t...|                    Ich war sehr enttäuscht über diesen Film|
|Translate English to German: I think vampire movies (usua...|Ich denke, dass die Vampire-Filme (normalerweise) schlech...|
|Translate English to German: Though not a complete waste ...|Obwohl es sich nicht um eine komplette Verschwendung von ...|
|Translate English to German: This film did well at the bo...|Dieser Film hat sich gut an der Boxoffice ereignet, und d...|
|Transla

In [29]:
# only use first 100 rows, since generation takes a while
df2 = df.withColumn("input", preprocess(col("lines"), "Translate English to French: ")).select("input").limit(100).cache()

In [30]:
df2.show(truncate=120)

+------------------------------------------------------------------------------------------------------------------------+
|                                                                                                                   input|
+------------------------------------------------------------------------------------------------------------------------+
|          Translate English to French: This is so overly clichéd you'll want to switch it off after the first 45 minutes|
|                                                      Translate English to French: I was very disappointed by this movie|
|                                                Translate English to French: I think vampire movies (usually) are wicked|
|Translate English to French: Though not a complete waste of time, 'Eighteen' really wasn't all sweet as it pretended ...|
|Translate English to French: This film did well at the box office, and the producers of this mess thought the stars h...|
|               

In [31]:
%%time
# first pass caches model/fn
preds = df2.withColumn("preds", generate(struct("input")))
result = preds.collect()

CPU times: user 3.96 ms, sys: 6.41 ms, total: 10.4 ms
Wall time: 11.4 s


In [32]:
%%time
preds = df2.withColumn("preds", generate("input"))
result = preds.collect()

CPU times: user 2.52 ms, sys: 5.26 ms, total: 7.78 ms
Wall time: 8.34 s


In [33]:
%%time
preds = df2.withColumn("preds", generate(col("input")))
result = preds.collect()

CPU times: user 4.62 ms, sys: 2.58 ms, total: 7.2 ms
Wall time: 8.29 s


In [34]:
preds.show(truncate=60)

+------------------------------------------------------------+------------------------------------------------------------+
|                                                       input|                                                       preds|
+------------------------------------------------------------+------------------------------------------------------------+
|Translate English to French: This is so overly clichéd yo...|                 Vous ne pouvez pas en tirer d'un tel cliché|
|Translate English to French: I was very disappointed by t...|                               Je suis très déçu par ce film|
|Translate English to French: I think vampire movies (usua...|Je pense que les films vampires (habituellement) sont méc...|
|Translate English to French: Though not a complete waste ...|    Bien qu'il ne soit pas un gaspillage complet de temps, '|
|Translate English to French: This film did well at the bo...|               Ce film a bien avancé à la salle de cinéma et|
|Transla

### Using Triton Inference Server

Note: you can restart the kernel and run from this point to simulate running in a different node or environment.  
While the examples above use Tensorflow, note that inference on the Triton server is run using PyTorch. PyTorch will be included in the tarball below, and no further environment changes are required by the user.

This notebook uses the [Python backend with a custom execution environment](https://github.com/triton-inference-server/python_backend#creating-custom-execution-environments) with the compatible versions of Python/Numpy for Triton 24.08, using a conda-pack environment created as follows:
```
conda create -n huggingface-tf -c conda-forge python=3.10.0
conda activate huggingface-tf

export PYTHONNOUSERSITE=True
pip install numpy==1.26.4 tensorflow[and-cuda] tf-keras transformers conda-pack 

conda-pack  # huggingface-tf.tar.gz
```

In [35]:
import os

In [38]:
%%bash
# copy custom model to expected layout for Triton
rm -rf models
mkdir -p models
cp -r models_config/hf_generation_tf models

# add custom execution environment
cp huggingface-tf.tar.gz models

#### Start Triton Server on each executor

In [39]:
num_executors = 1
triton_models_dir = "{}/models".format(os.getcwd())
huggingface_cache_dir = "{}/.cache/huggingface".format(os.path.expanduser('~'))
nodeRDD = sc.parallelize(list(range(num_executors)), num_executors)

def start_triton(it):
    import docker
    import time
    import tritonclient.grpc as grpcclient
    
    client=docker.from_env()
    containers=client.containers.list(filters={"name": "spark-triton"})
    if containers:
        print(">>>> containers: {}".format([c.short_id for c in containers]))
    else:
        try:
            container=client.containers.run(
                "nvcr.io/nvidia/tritonserver:24.08-py3", "tritonserver --model-repository=/models",
                detach=True,
                device_requests=[docker.types.DeviceRequest(device_ids=["0"], capabilities=[['gpu']])],
                environment=[
                    "TRANSFORMERS_CACHE=/cache"
                ],
                name="spark-triton",
                network_mode="host",
                remove=True,
                shm_size="1G",
                volumes={
                    triton_models_dir: {"bind": "/models", "mode": "ro"},
                    huggingface_cache_dir: {"bind": "/cache", "mode": "rw"}
                }
            )
            print(">>>> starting triton: {}".format(container.short_id))
        except Exception as e:
            print(">>>> failed to start triton: {}".format(e))
        # wait for triton to be running
        time.sleep(15)
        client = grpcclient.InferenceServerClient("localhost:8001")
        ready = False
        while not ready:
            try:
                ready = client.is_server_ready()
            except Exception as e:
                time.sleep(5)

    return [True]

nodeRDD.barrier().mapPartitions(start_triton).collect()

[True]

#### Run inference

In [40]:
import pandas as pd
from functools import partial
from pyspark.ml.functions import predict_batch_udf
from pyspark.sql.functions import col, pandas_udf, struct
from pyspark.sql.types import StringType

In [41]:
# only use first N examples, since this is slow
df = spark.read.parquet("imdb_test").limit(100).cache()

In [42]:
# only use first sentence and add prefix for conditional generation
def preprocess(text: pd.Series, prefix: str = "") -> pd.Series:
    @pandas_udf("string")
    def _preprocess(text: pd.Series) -> pd.Series:
        return pd.Series([prefix + s.split(".")[0] for s in text])
    return _preprocess(text)

In [43]:
# only use first 100 rows, since generation takes a while
df1 = df.withColumn("input", preprocess(col("lines"), "Translate English to German: ")).select("input").limit(100)

In [44]:
df1.show(truncate=120)

+------------------------------------------------------------------------------------------------------------------------+
|                                                                                                                   input|
+------------------------------------------------------------------------------------------------------------------------+
|          Translate English to German: This is so overly clichéd you'll want to switch it off after the first 45 minutes|
|                                                      Translate English to German: I was very disappointed by this movie|
|                                                Translate English to German: I think vampire movies (usually) are wicked|
|Translate English to German: Though not a complete waste of time, 'Eighteen' really wasn't all sweet as it pretended ...|
|Translate English to German: This film did well at the box office, and the producers of this mess thought the stars h...|
|               

In [45]:
def triton_fn(triton_uri, model_name):
    import numpy as np
    import tritonclient.grpc as grpcclient
    
    np_types = {
      "BOOL": np.dtype(np.bool_),
      "INT8": np.dtype(np.int8),
      "INT16": np.dtype(np.int16),
      "INT32": np.dtype(np.int32),
      "INT64": np.dtype(np.int64),
      "FP16": np.dtype(np.float16),
      "FP32": np.dtype(np.float32),
      "FP64": np.dtype(np.float64),
      "FP64": np.dtype(np.double),
      "BYTES": np.dtype(object)
    }

    client = grpcclient.InferenceServerClient(triton_uri)
    model_meta = client.get_model_metadata(model_name)
    
    def predict(inputs):
        if isinstance(inputs, np.ndarray):
            # single ndarray input
            request = [grpcclient.InferInput(model_meta.inputs[0].name, inputs.shape, model_meta.inputs[0].datatype)]
            request[0].set_data_from_numpy(inputs.astype(np_types[model_meta.inputs[0].datatype]))
        else:
            # dict of multiple ndarray inputs
            request = [grpcclient.InferInput(i.name, inputs[i.name].shape, i.datatype) for i in model_meta.inputs]
            for i in request:
                i.set_data_from_numpy(inputs[i.name()].astype(np_types[i.datatype()]))
        
        response = client.infer(model_name, inputs=request)
        
        if len(model_meta.outputs) > 1:
            # return dictionary of numpy arrays
            return {o.name: response.as_numpy(o.name) for o in model_meta.outputs}
        else:
            # return single numpy array
            return response.as_numpy(model_meta.outputs[0].name)
        
    return predict

In [46]:
generate = predict_batch_udf(partial(triton_fn, triton_uri="localhost:8001", model_name="hf_generation_tf"),
                             return_type=StringType(),
                             input_tensor_shapes=[[1]],
                             batch_size=100)

In [47]:
%%time
# first pass caches model/fn
preds = df1.withColumn("preds", generate(struct("input")))
results = preds.collect()

CPU times: user 9.6 ms, sys: 6.27 ms, total: 15.9 ms
Wall time: 3.11 s


In [48]:
%%time
preds = df1.withColumn("preds", generate("input"))
results = preds.collect()

CPU times: user 3.46 ms, sys: 342 μs, total: 3.8 ms
Wall time: 990 ms


In [49]:
%%time
preds = df1.withColumn("preds", generate(col("input")))
results = preds.collect()

CPU times: user 3.44 ms, sys: 522 μs, total: 3.96 ms
Wall time: 1.01 s


In [50]:
preds.show(truncate=60)

+------------------------------------------------------------+------------------------------------------------------------+
|                                                       input|                                                       preds|
+------------------------------------------------------------+------------------------------------------------------------+
|Translate English to German: This is so overly clichéd yo...|   Das ist so übertrieben klischeehaft, dass Sie es nach den|
|Translate English to German: I was very disappointed by t...|                    Ich war sehr enttäuscht über diesen Film|
|Translate English to German: I think vampire movies (usua...|Ich denke, dass die Vampire-Filme (normalerweise) schlech...|
|Translate English to German: Though not a complete waste ...|Obwohl es sich nicht um eine komplette Verschwendung von ...|
|Translate English to German: This film did well at the bo...|Dieser Film hat sich gut an der Boxoffice ereignet, und d...|
|Transla

In [51]:
# only use first 100 rows, since generation takes a while
df2 = df.withColumn("input", preprocess(col("lines"), "Translate English to French: ")).select("input").limit(100).cache()

24/10/03 01:09:57 WARN CacheManager: Asked to cache already cached data.


In [52]:
df2.show(truncate=120)

+------------------------------------------------------------------------------------------------------------------------+
|                                                                                                                   input|
+------------------------------------------------------------------------------------------------------------------------+
|          Translate English to French: This is so overly clichéd you'll want to switch it off after the first 45 minutes|
|                                                      Translate English to French: I was very disappointed by this movie|
|                                                Translate English to French: I think vampire movies (usually) are wicked|
|Translate English to French: Though not a complete waste of time, 'Eighteen' really wasn't all sweet as it pretended ...|
|Translate English to French: This film did well at the box office, and the producers of this mess thought the stars h...|
|               

In [53]:
%%time
preds = df2.withColumn("preds", generate(struct("input")))
results = preds.collect()

CPU times: user 2.5 ms, sys: 1.45 ms, total: 3.95 ms
Wall time: 1.31 s


In [54]:
%%time
preds = df2.withColumn("preds", generate("input"))
results = preds.collect()

CPU times: user 1.07 ms, sys: 2.05 ms, total: 3.13 ms
Wall time: 1.08 s


In [55]:
%%time
preds = df2.withColumn("preds", generate(col("input")))
results = preds.collect()

CPU times: user 1.15 ms, sys: 2.72 ms, total: 3.87 ms
Wall time: 996 ms


In [56]:
preds.show(truncate=60)

+------------------------------------------------------------+------------------------------------------------------------+
|                                                       input|                                                       preds|
+------------------------------------------------------------+------------------------------------------------------------+
|Translate English to French: This is so overly clichéd yo...|                 Vous ne pouvez pas en tirer d'un tel cliché|
|Translate English to French: I was very disappointed by t...|                               Je suis très déçu par ce film|
|Translate English to French: I think vampire movies (usua...|Je pense que les films vampires (habituellement) sont méc...|
|Translate English to French: Though not a complete waste ...|    Bien qu'il ne soit pas un gaspillage complet de temps, '|
|Translate English to French: This film did well at the bo...|               Ce film a bien avancé à la salle de cinéma et|
|Transla

#### Stop Triton Server on each executor

In [57]:
def stop_triton(it):
    import docker
    import time
    
    client=docker.from_env()
    containers=client.containers.list(filters={"name": "spark-triton"})
    print(">>>> stopping containers: {}".format([c.short_id for c in containers]))
    if containers:
        container=containers[0]
        container.stop(timeout=120)

    return [True]

nodeRDD.barrier().mapPartitions(stop_triton).collect()

[True]

In [58]:
spark.stop()